In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from pyha import *
from pyha.cores import *
from pyha.cores.util import *
import scipy
from scipy import signal
import warnings
warnings.filterwarnings('ignore') # yolo!
import numpy as np

# GitHub cannot display the Table of Contents!

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Info

Implementation is based on https://www.dsprelated.com/showarticle/58.php. This core implements the Dual-DC removal. The paper also discusses the Quad-DC removal, but my tests show it gives only slight improvement for alot of BRAM (only consider if you care about passband ripple..).
Works on 18-bit signed fixed-point numbers. Inputs and outputs assumed to be in range of -1 to 1. Outliers are saturated.

## Selection of ``window_len``
You want this to be infinite, as it makes the stopband sharper. OTOH this also controls how much BRAM you are using. 1024 is a nice value.

## On output width
This block outputs in 18-bit format, but usual usecase is to round it back to 12 bits (which is a common SDR size). Be warned that in this case DC **cannot** be removed if it is < ~80 dBm. I would suggest ~16 bits output for a reliable operation.

# Examples

## Remove DC component from complex signal

In [6]:
# file = get_data_file('dcspike_bladerf_20m.complex64')
file = get_data_file('from_tap.raw')
# file = get_data_file('gqrx_20180910_155357_2400499992_2999999_fc.raw')


input_signal = load_complex64_file(file) # IQ samples

dut = DCRemoval(window_len=256, dtype=Complex)
sims = simulate(dut, input_signal, trace=True, pipeline_flush='auto') # run simulations and gather trace
plot_trace()

# sim.plot_trace(mode={'Input': 'frequency', 'Output': 'frequency'})
# # sim.plot_trace(mode={'Input': 'frequency', 'Output': 'frequency'}, skip_first_n=1024*2)

TypeError: super(type, obj): obj must be an instance or subtype of type

## Frequency response

In [4]:
# NBVAL_IGNORE_OUTPUT
# taps 256 stopband  = -0.004 to 0.004
# taps 512 stopband  = -0.002 to 0.002
# taps 1024 stopband = -0.001 to 0.001
# reference one bin of 1024 point FFT is about 0.001, so 1024 taps clears 2 bins
input_signal = [0.0 + 0.0j] * 1024 * 16
input_signal[0] = 1.0 + 1.0j
dut = DCRemoval(window_len=1024, dtype=Complex)
sim = Simulator(dut, trace=True).run(input_signal)
sim.plot_trace(mode={'Input': 'time', 'Output': 'frequency_response'}, inout_only=True)

NameError: name 'Simulator' is not defined

# Conversion to VHDL and RTL/NETLIST simulations

In [ ]:
# Pyha design can be converted to VHDL after the PYHA simulation has been ran
# VHDL sources can be used to run the RTL simulation, in addition NETLIST simulation is supported (after Quartus mapper)
# Pyha automatically manages everything needed if you specify the 'RTL', 'NETLIST' flags.
output_dir = '~/Documents/pyha_output'
input_signal = (np.random.normal(size=1024) + np.random.normal(size=1024) * 1j) * 0.25
dut = DCRemoval(window_len=256)
vhdl_sim = Simulator(dut, output_dir=output_dir, extra_simulations=['RTL', 'NETLIST']).run(input_signal)
vhdl_sim.assert_equal(atol=1e-5, rtol=1e-5)

# Synthesis:  resource usage and Fmax

In [ ]:
print(vhdl_sim.quartus.get_resource_usage('fit'))
print(vhdl_sim.quartus.get_fmax())